<a href="https://colab.research.google.com/github/jidan-fikri/AWAN-App/blob/master/Awan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import layers

pre_trained_model = Xception(input_shape = (200, 200, 3), 
                                include_top = False, 
                                weights = 'imagenet')


for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()


In [ ]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (3, activation='softmax')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

In [ ]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam()
# Set the training parameters
model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

## Prepare the dataset

In [ ]:
!gdown --id 1f2zRPw1mDlkO_2Ze6mp8ecGu2S-LQkuO

In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator


with zipfile.ZipFile('cloud.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/cloud-classifier')

In [ ]:
base_dir = '/content/cloud-classifier'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 32,
                                                    class_mode = 'categorical', 
                                                    target_size = (200, 200))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 32,
                                                          class_mode  = 'categorical', 
                                                          target_size = (200, 200))

## Train the model

With that, you can now train the model. You will do 15 epochs and plot the results afterwards.

In [ ]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 15,
            verbose = 1)

## Evaluate the results

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

## Fine Tuning

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = True

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001)
# Set the training parameters
model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 15,
            verbose = 1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

## Save the Model and Convert it to TFLite

In [ ]:
CLOUD_SAVED_MODEL = "cloud_saved_model"

In [ ]:
tf.saved_model.save(model, CLOUD_SAVED_MODEL)

In [ ]:
%%bash -s $CLOUD_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(CLOUD_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

In [ ]:
!zip -r /content/saved_model.zip /content/cloud_saved_model

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)